Reading from the files and separating participants who completed the tasks from the rest.

In [1]:
import json

In [2]:
# Opening JSON file
f = open('output2.json')

In [3]:
data = json.load(f)

In [4]:
# for x in data['data']:
#     keys = x.keys()
#     print(keys)
#     values = x.values()
#     print(values)
#     print(x['completed'])
    

In [5]:
completed_participants=[]
for x in data['data']:
    if(x['completed']):
        completed_participants.append(x['mturk_id'])
# print(completed_participants)
        

In [6]:
f = open('output1.json')

In [7]:
jsonpmodel = json.load(f)

In [8]:
# for x in data['data']:
#     keys = x.keys()
#     print(keys)
#     print('\n')
#     values = x.values()
#     print(values)

In [9]:
completed_p_models=[]
for x in jsonpmodel['data']:
#     print(x['sequence_type'])
    m=x['mturk_id']
    if(m in completed_participants):
        completed_p_models.append(x)
        

In [10]:
filename='completedparticipants.json'
with open(filename, 'w') as json_file:
    json.dump(completed_p_models, json_file, 
                        indent=4,  
                        separators=(',',': '))

In [11]:
c_file=open(filename)
c_p=json.load(c_file)

In [12]:
# for x in c_p:
#     print(x['mturk_id'])

completedparticipants.json is the file that includes data of all participants who completed the task 
and c_p is the json varible of the file



In [13]:
# ----------------Experiment Sequences---------------------------

# N : first nbackn difficulty (H/E)

# Q : first Questionaire (1: NASA-TLX and 2: MRQ)

# S : SakeDifficulty(H/E)

# _________________________________________________________________________
# seq_number/Progress | 1.n1 | 2.s | 3.q1| 4.q2|5.n2 | 6.s |7.q1 |8.q2 |
# _________________________________________________________________________
#          1          |  E   | E   |  1  |  2  |  H  |  E  |  1  |  2  |
#          2          |  E   | E   |  2  |  1  |  H  |  E  |  2  |  1  |
#          3          |  H   | E   |  1  |  2  |  E  |  E  |  1  |  2  |
#          4          |  H   | E   |  2  |  1  |  E  |  E  |  2  |  1  |
#          5          |  E   | H   |  1  |  2  |  H  |  H  |  1  |  2  |
#          6          |  E   | H   |  2  |  1  |  H  |  H  |  2  |  1  |
#          7          |  H   | H   |  1  |  2  |  E  |  H  |  1  |  2  |
#          8          |  H   | H   |  2  |  1  |  E  |  H  |  2  |  1  |
# ________________________________________________________________________


In [14]:
 #  sequence type /progress matrix 
E_n = 1; # easy nbackn
H_n = 3; # hard nbackn
E_s = 500;  # easy sak in ms
H_s = 1000; #hard sak in ms
seq_prog = [
    [E_n, E_s, 1, 2, H_n, E_s, 1, 2],
    [E_n, E_s, 2, 1, H_n, E_s, 2, 1],
    [H_n, E_s, 1, 2, E_n, E_s, 1, 2],
    [H_n, E_s, 2, 1, E_n, E_s, 2, 1],
    [E_n, H_s, 1, 2, H_n, H_s, 1, 2],
    [E_n, H_s, 2, 1, H_n, H_s, 2, 1],
    [H_n, H_s, 1, 2, E_n, H_s, 1, 2],
    [H_n, H_s, 2, 1, E_n, H_s, 2, 1]
];

conE : Easy NBACKN----
conH : Hard NBACKN----
sakE: Easy SAK----
sakH: Hard SAK

In [15]:
nasa_1_t_array=[]
nasa_2_t_array=[]
nasa_1_r_array=[]
nasa_2_r_array=[]

seq_type_array=[]

mrq_1_array=[]
mrq_2_array=[]

mrq_conE_arry=[]
mrq_conH_arry=[]
nasa_conE_arry=[]
nasa_conH_arry=[]

mrq_sakE_arry=[]
mrq_sakH_arry=[]
nasa_sakE_arry=[]
nasa_sakH_arry=[]

for x in c_p:
    st= x['sequence_type']
    seq_type_array.append(st)
    
    ntr1= x['nasa_1_r'][0]
    ntr2= x['nasa_2_r'][0]
    ntt1= x['nasa_1_t'][0]
    ntt2= x['nasa_2_t'][0]
    
    nasa_1_r_array.append(ntr1)
    nasa_1_t_array.append(ntt1)
    nasa_2_r_array.append(ntr2)
    nasa_2_t_array.append(ntt2)
    
    
    s=x['mrq_1'][0].replace('"','').replace('[',"").replace(']',"").split(',')
    mrq1 = [int(y) for y in s]
    mrq1.pop()
    
    
    s=x['mrq_2'][0].replace('"','').replace('[',"").replace(']',"").split(',')
    mrq2 = [int(y) for y in s]
    mrq2.pop()
    
    mrq_1_array.append(mrq1)
    mrq_2_array.append(mrq2)
    
    if(st<=4):
    



IndentationError: expected an indented block (3419428986.py, line 50)

In [ ]:
for x in c_p:
    print(x['mrq_1'][0].replace('"','').replace('[',"").replace(']',"").split(','))
    x['sequence_type']
    x['mrq_1']
    x['mrq_2']
    x['nasa_1_r']
    x['nasa_1_t']
    x['nasa_2_r']
    x['nasa_2_t']
    